In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import numpy as np
import os 

# Suppress the TensorFlow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function to convert integer input to vectors for RNN using one hot encoding

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [3]:
# Generator function to generate infinite-stream of inputs for training

def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))

        Y = np.sort(X, axis=1)

        for ind,batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1

        yield x, y
        x.fill(0.0)
        y.fill(0.0)

In [4]:
def create_model(seq_len, max_no, n_layers, hidden_size):
    model = Sequential()
    # the encoder LSTM
    model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
    # in next layer, repeat the input seq_len times
    model.add(RepeatVector(seq_len))
    # decoder RNN, which will return output sequence
    for _ in range(n_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(max_no)))
    model.add(Activation('softmax'))
    return model

In [5]:
# Global parameters.
batch_size=32
seq_len = 15 # number of elements in sequence to sort
max_no = 100 # upper range of the numbers in sequence
hidden_size = 128
n_layers = 2
epochs = 100000

model = create_model(seq_len, max_no, n_layers, hidden_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(batch_gen(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 500 iteration to check how well it is performing
    if ind % 500 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = encode(testX, seq_len, max_no)
        #print(testX, ': input sequence')
        print(ind, ': epochs')
        print(acc, ': accuracy')
        loss_str = '{:4.3}'.format(loss)
        acc_str = '{:4.3}'.format(acc)
        
        y = model.predict(test, batch_size=1)
        np_sorted = np.sort(testX)[0]
        rnn_sorted = np.argmax(y, axis=2)[0]
        is_equal = np.array_equal(np_sorted, rnn_sorted)
        print(np_sorted, ': sorted by NumPy algorithm')
        print(rnn_sorted, ': sorted by trained RNN')
        print("\n")
        
        if acc > 0.995 :
            model.save('model.h5')
            break
        
    if ind > epochs:
        # Save trained model
        model.save('model.h5')
        break
    

(0, ': epochs')
(0.014583333, ': accuracy')
(array([ 6, 13, 33, 33, 41, 45, 52, 56, 73, 74, 82, 82, 92, 95, 99]), ': sorted by NumPy algorithm')
(array([59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59]), ': sorted by trained RNN')


(500, ': epochs')
(0.10625, ': accuracy')
(array([ 8, 18, 25, 29, 30, 37, 48, 64, 76, 81, 86, 89, 96, 96, 98]), ': sorted by NumPy algorithm')
(array([ 0, 13, 20, 29, 34, 41, 50, 58, 73, 80, 85, 88, 97, 99, 99]), ': sorted by trained RNN')


(1000, ': epochs')
(0.13125, ': accuracy')
(array([ 0,  3,  8, 15, 15, 30, 32, 37, 55, 58, 65, 73, 81, 95, 96]), ': sorted by NumPy algorithm')
(array([ 0,  6,  8, 14, 19, 27, 34, 41, 54, 64, 69, 75, 89, 96, 98]), ': sorted by trained RNN')


(1500, ': epochs')
(0.17083333, ': accuracy')
(array([ 1, 15, 22, 32, 38, 42, 42, 45, 47, 51, 56, 70, 82, 83, 90]), ': sorted by NumPy algorithm')
(array([ 0, 15, 27, 34, 37, 42, 44, 47, 51, 51, 57, 77, 82, 86, 91]), ': sorted by trained RNN')


(2000, ': epochs')
(0.225

(17000, ': epochs')
(0.7895833, ': accuracy')
(array([ 8, 14, 20, 32, 32, 35, 49, 53, 54, 74, 79, 83, 90, 95, 97]), ': sorted by NumPy algorithm')
(array([ 8, 14, 20, 31, 33, 34, 49, 53, 54, 74, 79, 83, 90, 95, 97]), ': sorted by trained RNN')


(17500, ': epochs')
(0.75208336, ': accuracy')
(array([ 0,  3, 17, 22, 23, 26, 30, 37, 41, 47, 55, 65, 68, 77, 99]), ': sorted by NumPy algorithm')
(array([ 0,  3, 17, 22, 24, 26, 30, 37, 40, 47, 55, 66, 68, 77, 99]), ': sorted by trained RNN')


(18000, ': epochs')
(0.82708335, ': accuracy')
(array([ 3,  7, 11, 15, 22, 23, 23, 38, 62, 64, 66, 72, 77, 88, 94]), ': sorted by NumPy algorithm')
(array([ 3,  7, 11, 16, 22, 23, 23, 38, 62, 64, 65, 72, 77, 88, 94]), ': sorted by trained RNN')


(18500, ': epochs')
(0.80625, ': accuracy')
(array([13, 18, 28, 28, 29, 43, 45, 47, 56, 61, 61, 90, 91, 96, 99]), ': sorted by NumPy algorithm')
(array([13, 18, 28, 28, 29, 43, 45, 47, 56, 61, 62, 90, 91, 96, 99]), ': sorted by trained RNN')


(19000, ': epoch

(34000, ': epochs')
(0.94375, ': accuracy')
(array([ 5, 33, 33, 38, 41, 44, 52, 56, 64, 68, 78, 82, 94, 97, 98]), ': sorted by NumPy algorithm')
(array([ 5, 33, 33, 38, 41, 44, 52, 56, 64, 68, 78, 82, 94, 97, 98]), ': sorted by trained RNN')


(34500, ': epochs')
(0.9770833, ': accuracy')
(array([ 2, 22, 23, 30, 39, 47, 59, 61, 73, 80, 83, 85, 90, 95, 98]), ': sorted by NumPy algorithm')
(array([ 2, 22, 23, 30, 39, 47, 59, 61, 73, 80, 83, 85, 90, 95, 98]), ': sorted by trained RNN')


(35000, ': epochs')
(0.97291666, ': accuracy')
(array([ 0, 29, 31, 31, 32, 33, 41, 44, 56, 61, 72, 81, 82, 91, 92]), ': sorted by NumPy algorithm')
(array([ 0, 29, 31, 31, 32, 33, 41, 44, 56, 61, 72, 81, 82, 91, 92]), ': sorted by trained RNN')


